In [1]:
import pyspark
import numpy as np

try:
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')

In [2]:
# bases = (id_base, nombre, pcia, ciudad, lat, lon)
# mediciones = (timestamp, id_base, temperatura, humedad, presion, direccion viento, velocidad_viento)

# Hallar por id y nombre las bases en Buenos Aires que hayan registrado una variacion de temperatura promedio mayor
# a 30% entre dos meses consecutivos

In [3]:
bases = [
    (1, 'Base Matanza', 'Buenos Aires', 'la Matanza', 110, 135),
    (2, 'Base Blanca', 'Buenos Aires', 'Bahia Blanca', 134, 190),
    (3, 'El Venadito', 'Santa Fe', 'Venado Tuerto', 220, 420)
]

mediciones = [
    ('2015-05-11', 1, 15, 23, 180, 'Norte', 22),
    ('2015-06-11', 1, 15.4, 24, 190, 'Noreste', 24),
    ('2011-02-15', 2, 4, 60, 320, 'Sureste', 36),
    ('2011-03-15', 2, 32, 75, 310, 'Sur', 20),
    ('2018-05-16', 3, 13, 43, 280, 'Norte', 16),
    ('2018-06-16', 3, 27, 45, 285, 'Norte', 19)
]

bases_rdd = sc.parallelize(bases)
mediciones_rdd = sc.parallelize(mediciones)

mediciones_rdd.take(5)

[('2015-05-11', 1, 15, 23, 180, 'Norte', 22),
 ('2015-06-11', 1, 15.4, 24, 190, 'Noreste', 24),
 ('2011-02-15', 2, 4, 60, 320, 'Sureste', 36),
 ('2011-03-15', 2, 32, 75, 310, 'Sur', 20),
 ('2018-05-16', 3, 13, 43, 280, 'Norte', 16)]

In [27]:
def diferencia_de_30(promedios_mensuales):
    for i in range (1, len(promedios_mensuales)):
        variacion_positiva = promedios_mensuales[i]/promedios_mensuales[i-1]
        variacion_negativa = promedios_mensuales[i-1]/promedios_mensuales[i]
        if (variacion_positiva >= 1.3 or variacion_negativa >= 1.3):
            return True
    return False
        

bases_bs_as = bases_rdd.map(lambda x: (x[0], (x[1], x[2])))\
             .filter(lambda x: x[1][1] == 'Buenos Aires')\
             .map(lambda x: (x[0], x[1][0]))

variacion_30 = mediciones_rdd.map(lambda x: (x[1], (x[0], x[2])))\
                             .join(bases_bs_as)\
                             .map(lambda x: ((x[0], x[1][1], x[1][0][0][0:7]), (x[1][0][1], 1)))\
                             .reduceByKey(lambda x,y: (x[0]+y[0], x[1], y[1]))\
                             .map(lambda x: ((x[0][0], x[0][1]), (x[1][0]/x[1][1], x[0][2])))\
                             .sortBy(lambda x: x[1][1])\
                             .reduceByKey(lambda x,y: (x[0],y[0]))\
                             .filter(lambda x: diferencia_de_30(x[1]))

variacion_30.take(5)

[((2, 'Base Blanca'), (4.0, 32.0))]